# Multiplication methods

### 1. Grid method

In the grid method, we split up the digits of each number and multiply them seperately, keeping track of the individual multiplications and the corresponding powers of ten. For this method we think of the numbers as strings, and perform single-digit multiplications using a table lookup. That is, we only multiply numbers using the "memorized" values of single digit multiplication, and thus our only computational task is to add up the products and distribute excesses to the correct powers of ten. (On paper we would use a grid to track the individual products, but this is not necessary in the algorithm.)

Example: Multiply 91 by 2.8

$91 \cdot 2.8$

= 

$ \ \ \ \ (9 \cdot 2)10^1 + (9 \cdot 8)10^0 $

$ + \ (1 \cdot 2)10^0 + (1 \cdot 8)10^{-1} $

=

$ (18)10^1 + (74)10^0 + (8)10^{-1} $

=

$ (25)10^1 + (4)10^0 + (8)10^{-1} $

=

$ (2)10^2 + (5)10^1 + (4)10^0 + (8)10^{-1} $

=

$254.8$


In [2]:
from collections import defaultdict
from operator import itemgetter


class Number:
    
    """Multiplication table for single digit numbers"""
    multTable = {
        str(d_one): {
            str(d_two): d_one*d_two for d_two in range(10)
        } 
        for d_one in range(10)
    }
    
    def __init__(self, string):
        if string[0] == '-':
            self.isNegative = True
            string = string[1:]
        else:
            self.isNegative = False
        string = string.strip('0')
        self.stringRep = string
        dotIndex = string.find('.')
        if dotIndex < 0:
            self.highestPower = len(string)-1
        else:
            self.highestPower = dotIndex-1
        self.digits = [d for d in string if d != '.']
        
    def __repr__(self):
        if self.isNegative:
            return '<Number: -' + self.stringRep + '>'
        return '<Number: ' + self.stringRep + '>'
    
    @classmethod
    def fromDigits(cls, digits, highestPower, negative=False):
        """Use list of digits and highest power to construct Number"""
        s = ''.join([str(digit) for digit in digits])
        if highestPower < 0:
            s = '.' + '0'*(-1-highestPower) + s
        else:
            dotIndex = highestPower + 1
            if dotIndex < len(digits):
                s = s[:dotIndex] + '.' + s[dotIndex:]
        if negative:
            s = '-' + s
        return cls(s)
    
    @property
    def digitPowers(self):
        """List of tuples containing digits and corresponding powers of ten"""
        
        return [(d, self.highestPower-i) for i, d in enumerate(self.digits)]
    
    def asFloat(self):
        if self.isNegative:
            return -1*float(self.stringRep)
        return float(self.stringRep)
    
    def __mul__(self, other):
        """Multiply two numbers by the grid method"""
        
        # 1. Compute products of individual digits, tracking powers of ten
        products = [
            (Number.multTable[selfDigit][otherDigit], selfPower + otherPower)
            for selfDigit, selfPower in self.digitPowers
            for otherDigit, otherPower in other.digitPowers
        ]
        productsByPower = defaultdict(int)
        for product, power in products:
            productsByPower[power] += product
        
        # 2. Loop over components, adding runover to the next digit until
        #    all components are in the range 0-9
        lowestPower = min(productsByPower.keys())
        highestPower = max(productsByPower.keys())
        currentPower = lowestPower
        while currentPower <= highestPower:
            currentProduct = productsByPower[currentPower]
            if currentProduct > 9:
                productsByPower[currentPower] = currentProduct % 10
                productsByPower[currentPower+1] += currentProduct // 10
                if currentPower == highestPower:
                    highestPower += 1
            currentPower += 1
        
        # 3. Construct and return the Number from the components
        components = list(productsByPower.items())
        components.sort(reverse=True, key=itemgetter(0))
        digits = [c[1] for c in components]
        hp = components[0][0]
        isNegative = (self.isNegative and not other.isNegative) or (not self.isNegative and other.isNegative)
        return Number.fromDigits(digits, highestPower=hp, negative=isNegative)
        
        
        
def gridMultiply(a, b):
    return (Number(str(a))*Number(str(b))).asFloat()

def compareMethods(a, b, multFn, fnName):
    """Print comparison of multFn and native multiplication"""
    
    fnProduct = multFn(a, b)
    nativeProduct = a*b
    print('Product of ' + str(a) + ' and ' + str(b))
    print('Native multiplication: ' + str(nativeProduct))
    print(fnName + ': ' + str(fnProduct))
    print('----')
    
def testMethod(a, b, multFn, fnName=None, pctError=10**(-8), verbose=False):
    """Test if multFn, native multiplication produce same result"""
    
    fnProduct = multFn(a, b)
    nativeProduct = a*b
    if verbose:
        print('Testing product of ' + str(a) + ' and ' + str(b))
        print('Native multiplication: {}'.format(nativeProduct))
        if fnName:
            print('{}: {}'.format(fnName, fnProduct))
        else:
            print('Alternate multiplication: {}'.format(fnProduct))
        print('----')
    return abs(nativeProduct - fnProduct)/nativeProduct < pctError

def runTests(testCases, multFn, fnName=None, pctError=10**(-8), verbose=False):
    """Run multiplication tests; return list of bools indicating tests passed"""
    
    testResults = [
        testMethod(a, b, multFn, fnName, pctError, verbose)
        for a, b in testCases
    ]
    if verbose:
        passed = sum(testResults)
        total = len(testResults)
        print('\nTest results:')
        print('Passed {} out of {} cases'.format(passed, total))
    return testResults

As = [12.56, 1.466, .09484, 48, 8]
Bs = [9.89, 245.256, .9834, 98.20001, 43]
testCases =  list(zip(As, Bs))
testCases += list(zip(As, reversed(Bs)))
testCases += list(zip(As, [-1*x for x in Bs]))
testCases += list(zip(reversed(As), [-1*x for x in Bs]))

testResults = runTests(testCases, multFn=gridMultiply, fnName='Grid method', verbose=True)

Testing product of 12.56 and 9.89
Native multiplication: 124.21840000000002
Grid method: 124.2184
----
Testing product of 1.466 and 245.256
Native multiplication: 359.545296
Grid method: 359.545296
----
Testing product of 0.09484 and 0.9834
Native multiplication: 0.093265656
Grid method: 0.093265656
----
Testing product of 48 and 98.20001
Native multiplication: 4713.60048
Grid method: 4713.60048
----
Testing product of 8 and 43
Native multiplication: 344
Grid method: 344.0
----
Testing product of 12.56 and 43
Native multiplication: 540.08
Grid method: 540.08
----
Testing product of 1.466 and 98.20001
Native multiplication: 143.96121466
Grid method: 143.96121466
----
Testing product of 0.09484 and 0.9834
Native multiplication: 0.093265656
Grid method: 0.093265656
----
Testing product of 48 and 245.256
Native multiplication: 11772.288
Grid method: 11772.288
----
Testing product of 8 and 9.89
Native multiplication: 79.12
Grid method: 79.12
----
Testing product of 12.56 and -9.89
Native mu

### 2. Quarter square multiplication

Quarter square multiplication is based on the following interesting algebraic identity:
    
$ xy = \frac{1}{4}(4xy) = \frac{1}{4}((x^2 + 2xy + y^2) - (x^2 - 2xy + y^2)) = \frac{(x + y)^2}{4} - \frac{(x - y)^2}{4} $

In fact,  when x and y are integers, we can replace $ \frac{(x + y)^2}{4} - \frac{(x - y)^2}{4} $ with
$ \frac{(x + y)^2}{4} - \frac{(x - y)^2}{4} $ because: if x+y, x-y are both even, then 4 divides them so the floor operator does nothing; and if x+y, x-y are both odd, then the remainders after dividing by 4 will cancel out due to the subtraction.

So, to simplify things, we will restrict to multiplying positive integers. Thanks to the identity, we can compute the product by doing a table lookup of (floors of) quarter squares. Such tables used to be published for this purpose, but we can generate our own table of quarter squares from 1 to 200,000, allowing us to multiply numbers up to 100,000 X 100,000.

In [6]:
class QSMultiplier():

    quarterSquareTable = {x: x**2 // 4 for x in range(200000)}
    
    def __init__(self):
        pass
    
    def multiply(self, x, y):
        a = x + y
        b = abs(x - y)
        return QSMultiplier.quarterSquareTable[a] - QSMultiplier.quarterSquareTable[b]


def QSMultiply(a, b):
    return QSMultiplier().multiply(a, b)

As = [43376, 41687, 55303, 6207, 29420, 19]
Bs = [12158, 58554, 342, 63819, 39481, 37334]
testFactors = list(zip(As, Bs)) + list(zip(As, reversed(Bs)))

testResults = runTests(testFactors, multFn=QSMultiply, fnName='Quarter square method', verbose=True)

Testing product of 43376 and 12158
Native multiplication: 527365408
Quarter square method: 527365408
----
Testing product of 41687 and 58554
Native multiplication: 2440940598
Quarter square method: 2440940598
----
Testing product of 55303 and 342
Native multiplication: 18913626
Quarter square method: 18913626
----
Testing product of 6207 and 63819
Native multiplication: 396124533
Quarter square method: 396124533
----
Testing product of 29420 and 39481
Native multiplication: 1161531020
Quarter square method: 1161531020
----
Testing product of 19 and 37334
Native multiplication: 709346
Quarter square method: 709346
----
Testing product of 43376 and 37334
Native multiplication: 1619399584
Quarter square method: 1619399584
----
Testing product of 41687 and 39481
Native multiplication: 1645844447
Quarter square method: 1645844447
----
Testing product of 55303 and 63819
Native multiplication: 3529382157
Quarter square method: 3529382157
----
Testing product of 6207 and 342
Native multiplicat

### 3. Karatsuba multiplication

If you had to multiply two numbers by hand, you would probably write both numbers down in rows, multiplying the digit pairs in your head, and adding up the results:

<pre>
    234510  
    x  998  
    ------
   1876080
  21105900
 211059000
 ---------
 234040980
</pre>

This is called long multiplication, or the standard algorithm. A weakness of this method is that if we have two n-digit numbers, then the time to multiply them by long multiplication is $O(n^2)$; i.e., the number of operations required scales at about the rate of the square of the size of the numbers. For extremely large numbers this can become a problem.

Karatsuba multiplication is a faster algorithm that requires $O(n^{\log_2 3}) \approx O(n^{1.585})$ operations. It achieves this speed increase by dividing the multiplication of two n-digit numbers into 3 multiplications involving numbers with less than n digits; then repeating that step on the three new multiplications, dividing them into multiplications involving even fewer digits, and so on, until finally we are only multiplying single-digit numbers. Because each step requires more addition/substraction operations than long multiplication, long multiplication is actually faster for small inputs. But for very large numbers (e.g. 1000 digits or more), the greatly reduced number of multiplications required ($n^{1.585}$ instead of $n^2$) makes Karatsuba multiplication much faster.

So how does the base step work? Suppose x and y are numbers specified as decimal strings of length n. Then let $m = \frac{n}{2}$. We can write $x = x_1 10^m + x_0$ and $y = y_1 10^m + y_0$, where $x_0$ and $y_0$ are less than $10^m$. Then the product can be written

$ xy = (x_1 10^m + x_0)(y_1 10^m + y_0) = (x_1 y_1) 10^{2m} + (x_0 y_1 + x_1 y_0) 10^{m} + x_0 y_0 $

To compute the right-hand side as it's written above, we still need to do 4 multiplications involving $x_0, x_1, y_0, y_1$. However we can reduce this to 3 by writing the middle coefficient in terms of the other two:

$x_0 y_1 + x_1 y_0 = (x_0 - x_1)(y_1 - y_0) + x_1 y_1 + x_0 y_0$

Now, we only have to do 3 multiplications involving shorter decimal strings, some additions/subtractions, and multiplications by powers of 10 (merely adding digits to the decimal strings). We can recursively compute the 3 multiplications, each time halving the lengths of the numbers being multiplied, resulting in a total number of multiplications of at most $n^{\log_2 3}$.

So the procedure consists of the following steps:
1. Write x, y as $x_1 10^m + x_0$ and $y_1 10^m + y_0$, such that $x_0, x_1, y_0, y_1$ are all less than $m$ digits.
2. Recursively compute the products $x_0 y_0$, $x_1 y_1$, and $(x_0 - x_1)(y_1 - y_0)$.
3. Use these values and some addition/subtraction operations to compute the middle coefficient $x_0 y_1 + x_1 y_0$
4. Finally use addition and multiplication by $10^m$ to compute $ (x_1 y_1) 10^{2m} + (x_0 y_1 + x_1 y_0) 10^{m} + x_0 y_0 = xy. $

In [28]:
class Number():
    """
    For storing and operating on integers represented by a
    decimal string and a sign bit
    """
    
    """Table of additions of numbers 0-10"""
    addTable = {
        str(d_one): {
            str(d_two): str(d_one + d_two) for d_two in range(11)
        } 
        for d_one in range(11)
    }
    
    """Table of products of single digit numbers"""
    multTable = {
        str(d_one): {
            str(d_two): str(d_one*d_two) for d_two in range(10)
        } 
        for d_one in range(10)
    }
    
    """Table of plus-ones for each digit < 9"""
    incTable = {str(d): str(d+1) for d in range(9)}
    
    def __init__(self, string, isNegative=False, preReversed=False):
        if not preReversed:
            string = string[::-1]
        string = string.rstrip('0')
        self.string = string
        self.isNegative = isNegative
        
    def __repr__(self):
        if self.isNegative:
            return '-' + self.string[::-1]
        return self.string[::-1]
    
    def asInt(self):
        return int(str(self))
        
    def __len__(self):
        return len(self.string)
    
    @staticmethod
    def addDigits(a, b, carry='0'):
        if carry == '0':
            return Number.addTable[a][b]
        return Number.addTable[a][ Number.addTable[b]['1'] ]
        
    def __add__(self, other):
        s = self.string
        o = other.string
        d = len(s) - len(o)
        if d > 0:
            o += '0'*d
        if d < 0:
            s += '0'*abs(d) 
            
        newDigits = []
        carry = '0'
        for ds, do in zip(s, o):
            r = Number.addDigits(ds, do, carry)
            if len(r) > 1:
                carry = '1'
            else:
                carry = '0'
            newDigits.append(r[-1])
        if carry == '1':
            newDigits.append('1')
        return Number(''.join(newDigits), preReversed=True)
    
    def __eq__(self, other):
        if self.isNegative != other.isNegative:
            return self.string == '0' and other.string == '0'
        return self.string.rstrip('0') == other.string.rstrip('0')
    
    def __lt__(self, other):
        """Return self < other"""
        
        if self.isNegative != other.isNegative:
            return self.isNegative
        if len(self) < len(other):
            return True
        if len(self) > len(other):
            return False
        
        # Numbers have same sign and equal lengths
        for i in range(len(self)-1, -1, -1):
            if self.string[i] < other.string[i]:
                return True
        return False
    
    @staticmethod
    def addOne(s, isReversed=True):
        """
        Add one to a raw decimal string. Removes need to create a Number
        object and call the lengthy add method, and so on.
        
        NOTE: default is reversed, i.e. least significant digit first
        """
        
        if isReversed:
            for i, d in enumerate(s):
                if d != '9':
                    result = ''.join([s[:i], Number.incTable[d], s[i+1:]])
        else:
            i = len(s) - 1
            while i > -1:
                if s[i] != '9':
                    return ''.join([s[:i], Number.incTable[s[i]], s[i+1:]])
                i -= 1

        # If here, then string was all 9s
        if isReversed:
            return ''.join(['1', '0'*len(s)])
        return ''.join(['0'*len(s), '1'])
            
    
    def __sub__(self, other):
        """Compute self-other by method of ten's complement"""
        
        # 1. Pad digits with zeros to get same length
        
        # 2. Note sign of result (use it at the end)
        
        # 3. Get nine's complement of y
        
        # 4. Compute s = x + (9's comp)(y) + 1
        
        # 5. Compute ten's complement of s
        
        # 6. Return (10's comp)(s) with appropriate sign
    
    

print(Number.addOne('349999', isReversed=False))    
    
        

class Karatsuba():
    
    def __init__(self):
        pass
    
    def multiply(self, x, y):
        """Multiply decimal strings x, y by Karatsuba multiplication"""
        
        if len(x) < 2 or len(y) < 2:
            return self.singleDigitMultiply(x, y)

359999
